## Jupyter notebook to generate silicon clusters with roughly the same density between different ``n-mers''

In [ ]:
import numpy as np
import ase

from skmatter.sample_selection import FPS
from tqdm.notebook import tqdm
from ase.io import read, write

### utility functions

In [ ]:
def get_growvec(rmin, rmax):
    
    phi = np.random.uniform(0, np.pi*2)
    costheta = np.random.uniform(-1, 1)
    u = np.random.uniform(0,1)
    r = rmin + np.power(u, 1/3) * (rmax - rmin)
    
    theta = np.arccos(costheta)
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)

    return np.array([x,y,z])
    

In [ ]:
def min_distance(l):
    # gets the minimum inter-particle distance in a point cloud
    l2 = (l**2).sum(axis=1)
    d2 = np.add.outer(l2,l2)-2*l@l.T
    np.fill_diagonal(d2, np.inf)
    return np.min(d2)

In [ ]:
def max_distance(l):
    # gets the minimum inter-particle distance in a point cloud
    l2 = (l**2).sum(axis=1)
    d2 = np.add.outer(l2,l2)-2*l@l.T
    np.fill_diagonal(d2, 0)
    return np.max(d2)
    

In [ ]:

def grow_cluster(n, bmin, bmax, bcmax, dmax):
    
    ## grow the cluster, place initial atom at origin
    l = np.array([[0,0,0]])
    cur_bc = np.array([0])
    finished = False
    
    while len(l) != n:

        for j in range(10):
            try_l = np.vstack([l.copy(), l[np.random.randint(len(l))] + get_growvec(bmin, bmax)])
            mind = min_distance(try_l)
            maxd = max_distance(try_l)
            cur_bcmax = ((ase.Atoms("Si{}".format(len(try_l)), try_l).get_all_distances() < bmax+0.25).sum(axis=0) - 1).max()
            
            if mind>bmin**2 and maxd<dmax**2 and cur_bcmax <= bcmax:
                l = try_l
                break
                
            if j == 9:
                l = np.delete(l, -1, 0)
        
    return l
    

In [ ]:
def make_unique(cls):
    # zeroes the center of mass
    cls = cls - cls.mean(axis=0)
    # first, sort according to the distance from the COM to remove permutation degeneracy
    order = np.argsort((cls[:]**2).sum(axis=1))[::-1]
    # and now we apply an orthogonal transformation that makes the i-th vector have all
    # the coordinates above the i-th zero (this basically sets the rotations)
    q,r = np.linalg.qr(cls[order].T)
    # now set the orientation along the axes
    for i in range(min(r.shape)):
        r[i] *= np.sign(r[i,i])    
    return r.T
    

### density consideration

In [ ]:
## 4.121 is the distance (not radius) at which dimer cohesive energy is at 25% of "minimum"
dimer_volume = 4/3*np.pi*(4.121/2)**3

In [ ]:
dimer_density = 2 / dimer_volume

In [ ]:

body_list = np.arange(3,11)
const_dens_radii = np.power(3/4/(np.pi)*(dimer_volume/2 * body_list), 1/3)


### cluster generation

Below generates n-mers from n = 3 to 10.

In [ ]:

np.random.seed(1215)

all_cls = []
for na in range(3, 11):
    cls = []
    for i in tqdm(range(10000)):
        cls.append(make_unique(grow_cluster(na, 2.0, 2.5, 6, const_dens_radii[na-3]*2.0)))
    cls = np.asarray(cls)
    all_cls.append(cls)
    

In [ ]:

all_frames = []
for ii in range(8):
    frames = [ase.Atoms("Si{}".format(ii+3), all_cls[ii][i]) for i in range(len(all_cls[ii]))]
    for f in frames:
        f.info['config_type'] = str(ii+3) + 'mer'
    all_frames.append(frames)


In [ ]:

all_sel_cls = []
for ii in range(8):
    selector = FPS(n_to_select=1000)
    selector.fit(all_cls_5[ii].reshape(10000,-1))
    all_sel_cls.append(all_cls_5[ii][selector.selected_idx_])    


In [ ]:

all_sel_frames = []
for ii in range(8):
    frames = [ase.Atoms("Si{}".format(ii+3), all_sel_cls[ii][i]) for i in range(len(all_sel_cls[ii]))]
    for f in frames:
        f.info['config_type'] = str(ii+3) + 'mer'    
    all_sel_frames.append(frames)
    write("Si_cluster_{}.xyz".format(ii+3), all_sel_frames[ii])    
